In [1]:
# ! pip install bs4
# ! pip install contractions
# ! pip install gensim

In [2]:
# pip install nltk

In [3]:
import pandas as pd
import numpy as np
import nltk
import warnings
warnings.filterwarnings("ignore")
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import re
from bs4 import BeautifulSoup
import contractions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim.downloader as api
import gensim.models
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import gc
from sys import getsizeof

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/ayanpatel_69/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ayanpatel_69/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/ayanpatel_69/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Task 1: Dataset Generation

In [4]:
df = pd.read_csv('./data.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)
df = df[['star_rating', 'review_body']]
class_one = df[(df['star_rating']==1) | (df['star_rating']==2)]
class_two = df[df['star_rating']==3]
class_three = df[(df['star_rating']==4) | (df['star_rating']==5)]

class_one.loc[:, "label"] =1
class_two.loc[:, "label"] =2
class_three.loc[:, "label"] =3

class_one = class_one.sample(n=20000, random_state=100)
class_two = class_two.sample(n=20000, random_state=100)
class_three = class_three.sample(n=20000, random_state=100)
# dataset = pd.concat([class_one, class_two, class_three])
df = pd.concat([class_one, class_two, class_three])

df.reset_index(drop=True)
train = df.sample(frac=0.8, random_state=100)
test = df.drop(train.index)

train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

# train.to_csv('train.csv', index = False)
# test.to_csv('test.csv', index = False)

del globals()['class_one'], globals()['class_two'], globals()['class_three'], globals()['df']
df = dataset = [[99999, 99999]]
del df, dataset
gc.collect()

0

In [5]:
# Pretrained Word2Vec model:
pretrained_w2v = api.load('word2vec-google-news-300')

In [6]:
gc.collect()

56

In [7]:
print('Check semantic similarities of the generated vectors:')
print(pretrained_w2v.most_similar(positive=['king', 'woman'], negative=['man'], topn = 1))
print('Excellent ~ Outstanding:', pretrained_w2v.similarity('excellent', 'outstanding'))
print('time ~ schedule:', pretrained_w2v.similarity('time', 'schedule'))

Check semantic similarities of the generated vectors:
[('queen', 0.7118193507194519)]
Excellent ~ Outstanding: 0.5567486
time ~ schedule: 0.26993576


In [8]:
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')

In [9]:
# Covert all reviews to lower case
train['review_body'] = train['review_body'].str.lower()
test['review_body'] = test['review_body'].str.lower()

'''
URL Remover code
'''
train['review_body'] = train['review_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
test['review_body'] = test['review_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

def html_tag_remover(review):
    soup = BeautifulSoup(review, 'html.parser')
    review = soup.get_text()
    return review  

train['review_body'] = train['review_body'].apply(lambda review: html_tag_remover(review))
test['review_body'] = test['review_body'].apply(lambda review: html_tag_remover(review))

'''
remove non-alphabetical characters
'''
train['review_body'] = train['review_body'].apply(lambda review: re.sub('[^a-zA-Z]+',' ', review))
test['review_body'] = test['review_body'].apply(lambda review: re.sub('[^a-zA-Z]+',' ', review))

'''
remove extra spaces
'''
train['review_body'] = train['review_body'].apply(lambda review: re.sub(' +', ' ', review))
test['review_body'] = test['review_body'].apply(lambda review: re.sub(' +', ' ', review))

'''
perform contractions on the reviews
'''
def expand_contractions(review):
    review = contractions.fix(review)
    return review

train['review_body'] = train['review_body'].apply(lambda review: expand_contractions(review))
test['review_body'] = test['review_body'].apply(lambda review: expand_contractions(review))

In [10]:
'''
remove the stop words AND perform lemmatization

'''
def remove_stopwords(review):
    stop_words_english = set(stopwords.words('english'))
    review_word_tokens = word_tokenize(review)
    filtered_review = [word for word in review_word_tokens if not word in stop_words_english]
    return filtered_review

train['review_body'] = train['review_body'].apply(lambda review: remove_stopwords(review))
test['review_body'] = test['review_body'].apply(lambda review: remove_stopwords(review))

def review_lemmatize(review):
    lemmatizer = WordNetLemmatizer()
    lemmatized_review = [lemmatizer.lemmatize(word) for word in review]
    return ' '.join(lemmatized_review)    

train['review_body'] = train['review_body'].apply(lambda review: review_lemmatize(review))
test['review_body'] = test['review_body'].apply(lambda review: review_lemmatize(review))

# Task 2: Word Embedding

In [11]:
all_Sentences = [sentence.split(' ') for sentence in train['review_body'].to_list()]

In [12]:
# Custom Word2Vec
custom_model = gensim.models.Word2Vec(all_Sentences, vector_size = 300, min_count=9, window=13)

In [13]:
print('Check semantic similarities of the generated vectors:')
print(custom_model.wv.most_similar(positive=['king', 'woman'], negative=['man'], topn = 1)[0])
print('Excellent ~ Outstanding:', custom_model.wv.similarity('excellent', 'outstanding'))
print('time ~ schedule:', custom_model.wv.similarity('time', 'schedule'))

Check semantic similarities of the generated vectors:
('touted', 0.7405036687850952)
Excellent ~ Outstanding: 0.68179345
time ~ schedule: 0.21108943


In [14]:
del all_Sentences, custom_model
gc.collect()
all_Sentences = [1]
custom_model = [1]

# Task 3: Simple Models

In [15]:
# Average word2Vec vectors
def average_vectors(review, label):
    temp_review = review.split(' ')
    review_vector = np.array([pretrained_w2v[word] for word in temp_review if word in pretrained_w2v])
    if len(review_vector) >=1:
#         review_vector = []
#         for word in words:
#             review_vector.append(pretrained_w2v[word])
        return review_vector, label

def average_vectors_concat(review, label):
    temp_review = review.split(' ')
    review_vector = np.array([pretrained_w2v[word] for word in temp_review[:10] if word in pretrained_w2v])
    
    # can be the case where the words in the review are not found in the W2V vocabulary
    if len(review_vector)==0:
        review_vector = np.zeros((1, 300))
    review_vector = np.concatenate(review_vector, axis=0)
    
    # In the case where the total dim of the feature vector is <3000 add the padding with zeros
    if len(review_vector)<3000:
        review_vector = np.concatenate([review_vector, np.zeros(3000-len(review_vector))])
    return review_vector/10, label    
    
def featurization(dataset, concat = False):
    features = []
    y_labels = []
    concat = concat
    
    for review, label in zip(dataset['review_body'], dataset['label']):
        try:
            if not concat:
                x, y = average_vectors(review, label)
                features.append(np.mean(x, axis=0))
            else:
                x, y = average_vectors_concat(review, label)
                features.append(x)
                
            y_labels.append(y)
        
        except:
            pass
    return features, y_labels

# Vectors without concatenation
w2v_pretrain_train_x, w2v_pretrain_train_y = featurization(train)
w2v_pretrain_test_x, w2v_pretrain_test_y = featurization(test)

# Vectors with concatenation
w2v_pretrain_train_concat_x, w2v_pretrain_train_concat_y = featurization(train, True)
w2v_pretrain_test_concat_x, w2v_pretrain_test_concat_y = featurization(test, True)

In [16]:
# TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer(min_df = 0.001)
tfidf_X_train = tfidf_vectorizer.fit_transform(list(train['review_body']))
tfidf_X_train = pd.DataFrame(tfidf_X_train.toarray())

tfidf_X_test = tfidf_vectorizer.transform(list(test['review_body']))
tfidf_X_test = pd.DataFrame(tfidf_X_test.toarray())

tfidf_Y_train = train['label']
tfidf_Y_test = test['label']

tfidf_Y_train = tfidf_Y_train.astype('int')
tfidf_Y_test = tfidf_Y_test.astype('int')

In [17]:
# Training Perceptron Model on Average Word2Vec Features
perceptr_w2v = Perceptron(random_state = 100, eta0=0.1)
perceptr_w2v.fit(w2v_pretrain_train_x, w2v_pretrain_train_y)
Y_pred_w2v_test = perceptr_w2v.predict(w2v_pretrain_test_x)

# Training Perceptron Model on TF-IDF Features
perceptr_tfidf = Perceptron(random_state = 100, eta0=0.1)
perceptr_tfidf.fit(tfidf_X_train, tfidf_Y_train)
Y_pred_tfidf_test = perceptr_tfidf.predict(tfidf_X_test)

target_names = ['class 1', 'class 2', 'class 3']
report_w2v = classification_report(w2v_pretrain_test_y, Y_pred_w2v_test, target_names=target_names, output_dict=True)
report_tfidf = classification_report(tfidf_Y_test, Y_pred_tfidf_test, target_names=target_names, output_dict=True)

In [18]:
print('Accuracy values PERCEPTRON for w2v and tfidf features:')
print(report_w2v['accuracy'], report_tfidf['accuracy'])

Accuracy values PERCEPTRON for w2v and tfidf features:
0.5805374728759807 0.6170833333333333


In [19]:
# Training SVM Model on Average Word2Vec Features
svm_w2v = LinearSVC(random_state=100, max_iter=1000)
svm_w2v.fit(w2v_pretrain_train_x, w2v_pretrain_train_y)
Y_pred_w2v_svm_test = svm_w2v.predict(w2v_pretrain_test_x)

# Training SVM Model on TFIDF Features
svm_tfidf = LinearSVC(random_state=100, max_iter=1000)
svm_tfidf.fit(tfidf_X_train, tfidf_Y_train)
Y_pred_tfidf_svm_test = svm_tfidf.predict(tfidf_X_test)

report_svm_w2v = classification_report(w2v_pretrain_test_y, Y_pred_w2v_svm_test, target_names=target_names, output_dict=True)
report_svm_tfidf = classification_report(tfidf_Y_test, Y_pred_tfidf_svm_test, target_names=target_names, output_dict=True)

In [20]:
print('Accuracy values SVM for w2v and tfidf features:')
print(report_svm_w2v['accuracy'], report_svm_tfidf['accuracy'])

Accuracy values SVM for w2v and tfidf features:
0.627691537305959 0.6685


# Task 4: Feedforward Neural Networks

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset

In [22]:
device = torch.device('cpu')

In [23]:

class MLP(nn.Module):
    def __init__(self, classification = "binary", vocab_size = 300):
        super(MLP, self).__init__()
        hidden_1 = 100
        hidden_2 = 10
        self.fc1 = nn.Linear(vocab_size, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        if classification == "binary":
            self.fc3 = nn.Linear(hidden_2, 3)
        else:
            # For multi-classification
            self.fc3 = nn.Linear(hidden_2, 4)  
        
#         self.sig = nn.Sigmoid()
#         self.soft = nn.Softmax(dim = 1)
    
    
    def forward(self, x):
        x = x.view(-1, x.shape[1])
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
class MLP_concat(nn.Module):
    def __init__(self, classification = "binary", vocab_size = 3000):
        super(MLP_concat, self).__init__()
        hidden_1 = 100
        hidden_2 = 10
        self.fc1 = nn.Linear(vocab_size, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        if classification == "binary":
            self.fc3 = nn.Linear(hidden_2, 3)
        else:
            # For multi-classification
            self.fc3 = nn.Linear(hidden_2, 4)  
        
#         self.sig = nn.Sigmoid()
#         self.soft = nn.Softmax(dim = 1)
    
    
    def forward(self, x):
        x = x.view(-1, x.shape[1])
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = MLP()
model_concat = MLP_concat()
model = model
model_concat = model_concat
print(model)
print(model_concat)

MLP(
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
)
MLP_concat(
  (fc1): Linear(in_features=3000, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
)


-- Task 4(a) using the average Word2Vec vectors

In [24]:
train_data=TensorDataset(torch.FloatTensor(w2v_pretrain_train_x), torch.LongTensor(w2v_pretrain_train_y))
test_data=TensorDataset(torch.FloatTensor(w2v_pretrain_test_x), torch.LongTensor(w2v_pretrain_test_y))

# Data Loader
train_batch_size=16
train_loader=DataLoader(train_data, batch_size=train_batch_size, shuffle=True)

test_batch_size=16
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
criterion = criterion
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# number of epochs to train the model
n_epochs = 20

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity
best_acc = 0

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    # train the model #
    model.train() # prep model for training
    for data, target in train_loader: # iterates upto number of batch size
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, (target-1))
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
          
    # validate the model #
    model.eval() # prep model for evaluation
    correct = 0
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, (target-1))
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        ypred = output.argmax(dim = 1)
        correct += (ypred == (target-1)).float().sum()
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tEpoch Accuracy: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss,
        correct/len(test_loader.dataset)
        ))

Epoch: 1 	Training Loss: 1.097732 	Validation Loss: 1.095251 	Epoch Accuracy: 0.373811
Epoch: 2 	Training Loss: 1.086353 	Validation Loss: 1.067464 	Epoch Accuracy: 0.441662
Epoch: 3 	Training Loss: 1.001967 	Validation Loss: 0.955275 	Epoch Accuracy: 0.533133
Epoch: 4 	Training Loss: 0.916183 	Validation Loss: 0.906085 	Epoch Accuracy: 0.559923
Epoch: 5 	Training Loss: 0.884873 	Validation Loss: 0.890566 	Epoch Accuracy: 0.574779
Epoch: 6 	Training Loss: 0.867173 	Validation Loss: 0.871425 	Epoch Accuracy: 0.587298
Epoch: 7 	Training Loss: 0.851008 	Validation Loss: 0.853370 	Epoch Accuracy: 0.605241
Epoch: 8 	Training Loss: 0.836682 	Validation Loss: 0.841987 	Epoch Accuracy: 0.616341
Epoch: 9 	Training Loss: 0.826298 	Validation Loss: 0.859752 	Epoch Accuracy: 0.607745
Epoch: 10 	Training Loss: 0.818826 	Validation Loss: 0.833793 	Epoch Accuracy: 0.625104
Epoch: 11 	Training Loss: 0.813659 	Validation Loss: 0.832400 	Epoch Accuracy: 0.625188
Epoch: 12 	Training Loss: 0.809042 	Valid

-- Test Dataset Accuracy classwise

In [25]:
model.eval() # prep model for evaluation
main_tar = []
predss = []
with torch.no_grad():
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, (target-1))
        ypred = output.argmax(dim = 1)
        for i in np.array(target-1):
            main_tar.append(i)
        for j in np.array(ypred):
            predss.append(j)

              precision    recall  f1-score   support

           0       0.60      0.77      0.67      4085
           1       0.60      0.43      0.50      3979
           2       0.69      0.68      0.69      3918

    accuracy                           0.63     11982
   macro avg       0.63      0.63      0.62     11982
weighted avg       0.63      0.63      0.62     11982



In [26]:
print("Accuracy Value")
report = classification_report(main_tar, predss, digits=6, output_dict=True)
print(report['accuracy'])

Accuracy Value
0.6290268736437991


## Accuracy Value for FNN: 0.635

Task 4(b) 10 word vectors concatenated

In [27]:

train_data=TensorDataset(torch.FloatTensor(w2v_pretrain_train_concat_x), torch.LongTensor(w2v_pretrain_train_concat_y))

test_data=TensorDataset(torch.FloatTensor(w2v_pretrain_test_concat_x), torch.LongTensor(w2v_pretrain_test_concat_y))

# Data Loader
train_batch_size=16
train_loader=DataLoader(train_data, batch_size=train_batch_size, shuffle=True)

test_batch_size=16
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
criterion = criterion
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.Adam(model_concat.parameters(), lr=0.002)

# number of epochs to train the model
n_epochs = 10

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity
best_acc = 0

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    # train the model #
    model_concat.train() # prep model for training
    for data, target in train_loader: # iterates upto number of batch size
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_concat(data)
        # calculate the loss
        loss = criterion(output, target-1)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
          
    # validate the model #
    model_concat.eval() # prep model for evaluation
    correct = 0
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_concat(data)
        # calculate the loss
        loss = criterion(output, target-1)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        ypred = output.argmax(dim = 1)
        correct += (ypred == target-1).float().sum()
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tEpoch Accuracy: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss,
        correct/len(test_loader.dataset)
        ))

Epoch: 1 	Training Loss: 0.932305 	Validation Loss: 0.894321 	Epoch Accuracy: 0.574167
Epoch: 2 	Training Loss: 0.849281 	Validation Loss: 0.889959 	Epoch Accuracy: 0.576417
Epoch: 3 	Training Loss: 0.781346 	Validation Loss: 0.913633 	Epoch Accuracy: 0.568500
Epoch: 4 	Training Loss: 0.685194 	Validation Loss: 0.987501 	Epoch Accuracy: 0.557917
Epoch: 5 	Training Loss: 0.564092 	Validation Loss: 1.133868 	Epoch Accuracy: 0.546250
Epoch: 6 	Training Loss: 0.442663 	Validation Loss: 1.349558 	Epoch Accuracy: 0.536333
Epoch: 7 	Training Loss: 0.332685 	Validation Loss: 1.615199 	Epoch Accuracy: 0.532917
Epoch: 8 	Training Loss: 0.251585 	Validation Loss: 1.988344 	Epoch Accuracy: 0.528583
Epoch: 9 	Training Loss: 0.198705 	Validation Loss: 2.301472 	Epoch Accuracy: 0.523417
Epoch: 10 	Training Loss: 0.170449 	Validation Loss: 2.616928 	Epoch Accuracy: 0.523167


In [28]:
model_concat.eval() # prep model for evaluation
main_tar = []
predss = []
with torch.no_grad():
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_concat(data)
        # calculate the loss
        loss = criterion(output, target-1)
        # update running validation loss 
        #valid_loss += loss.item()*data.size(0)
        ypred = output.argmax(dim = 1)
        for i in np.array(target-1):
            main_tar.append(i)
        for j in np.array(ypred):
            predss.append(j)
        
# print("Accuracy Values for each class")
# print(classification_report(main_tar, predss))

In [29]:
print("Accuracy Value: After contenating first 10 review vectors")
concate_report = classification_report(main_tar, predss, digits=6, output_dict=True)
print(concate_report['accuracy'])

Accuracy Value: After contenating first 10 review vectors
0.5231666666666667


In [30]:
del globals()['tfidf_X_train'], globals()['tfidf_X_test'], globals()['tfidf_Y_train'], globals()['tfidf_Y_test']

del globals()['w2v_pretrain_train_x'], globals()['w2v_pretrain_train_y']
del globals()['w2v_pretrain_test_x'], globals()['w2v_pretrain_test_y']

del globals()['w2v_pretrain_train_concat_x'], globals()['w2v_pretrain_train_concat_y']
del globals()['w2v_pretrain_test_concat_x'], globals()['w2v_pretrain_test_concat_y']

del globals()['model'], globals()['model_concat'], globals()['train_data'], globals()['test_data']
del globals()['Y_pred_w2v_test'], globals()['Y_pred_tfidf_test'], globals()['Y_pred_w2v_svm_test'], globals()['Y_pred_tfidf_svm_test']

del globals()['train_loader'], globals()['test_loader']
del globals()['main_tar'], globals()['predss']



gc.collect()

63

# Task 5 Recurrent Neural Networks

In [46]:
'''
limiting the maximum review length to 20 by truncating longer reviews and padding
shorter reviews with a null value (0)
'''
# Average word2Vec vectors
def average_vectors_rnn(review):
    temp_review = review.split(' ')
        
#     review_vector = []
#     for word in words:
#         review_vector.append()
#     review_vector = np.array(review_vector)

    review_vector = np.array([pretrained_w2v[word] for word in temp_review[:20] if word in pretrained_w2v])
    
    # can be the case where the words in the review are not found in the W2V vocabulary
    if len(review_vector)==0:
        review_vector = np.zeros((20, 300))
    
    # In the case where the total dim of the feature vector is <20 add the padding with zeros
    elif len(review_vector)<20:
        review_vector = np.concatenate([review_vector, np.zeros((20-len(review_vector), 300))])
        
    return review_vector
    
def featurization_rnn(dataset):
    features = []

    for review in dataset['review_body']:
        x = average_vectors_rnn(review)
        features.append(x)
        
    return features

# Review Vectors for  first 20 words each
w2v_pretrain_train_x = featurization_rnn(train)
w2v_pretrain_train_y = train['label']
w2v_pretrain_test_x = featurization_rnn(test)
w2v_pretrain_test_y = test['label']



- Task 5(a): Train a simple RNN for sentiment analysis.


In [61]:
class rnn_model(nn.Module):
    def __init__(self):
        super(rnn_model, self).__init__()
        
        self.rnn_layer = nn.RNN(300, 20, batch_first = True)        
        self.fc = nn.Linear(20,3)

    def forward(self, input):
        output = input.view(-1,20,300)
        output, hidden = self.rnn_layer(output)
        output=self.fc(output[:,-1,:])
        return output

model = rnn_model()
print(model)

rnn_model(
  (rnn_layer): RNN(300, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=3, bias=True)
)


In [48]:
train_data=TensorDataset(torch.FloatTensor(w2v_pretrain_train_x), torch.LongTensor(w2v_pretrain_train_y))
test_data=TensorDataset(torch.FloatTensor(w2v_pretrain_test_x), torch.LongTensor(w2v_pretrain_test_y))

# Data Loader
train_batch_size=256
train_loader=DataLoader(train_data, batch_size=train_batch_size, shuffle=True)

test_batch_size=256
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)

In [62]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
criterion = criterion
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# number of epochs to train the model
n_epochs = 2

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity
best_acc = 0

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    # train the model #
    model.train() # prep model for training
    for data, target in train_loader: # iterates upto number of batch size
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target-1)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
          
    # validate the model #
    model.eval() # prep model for evaluation
    correct = 0
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target-1)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        ypred = output.argmax(dim = 1)
        correct += (ypred == target-1).float().sum()
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tEpoch Accuracy: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss,
        correct/len(test_loader.dataset)
        ))
    
model.eval() # prep model for evaluation
main_tar = []
predss = []
with torch.no_grad():
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target-1)
        ypred = output.argmax(dim = 1)
        for i in np.array(target-1):
            main_tar.append(i)
        for j in np.array(ypred):
            predss.append(j)

Epoch: 1 	Training Loss: 1.059479 	Validation Loss: 0.980146 	Epoch Accuracy: 0.513750
Epoch: 2 	Training Loss: 0.953794 	Validation Loss: 0.935519 	Epoch Accuracy: 0.541917


In [35]:
print("Accuracy Value: RNN")
rnn_report = classification_report(main_tar, predss, digits=6, output_dict=True)
print(rnn_report['accuracy'])

Accuracy Value: RNN
0.6061666666666666


Task 5(b): Considering a gated recurrent unit cell.

In [36]:
class gru_model(nn.Module):
    def __init__(self):
        super(gru_model, self).__init__()
        self.gru_layer = nn.GRU(300, 20, batch_first = True)
        self.fc = nn.Linear(20,3)

    def forward(self, input):
        output = input.view(-1,20,300)
        output, hidden = self.gru_layer(output)
        output=self.fc(output[:,-1,:])
        return output

model_gru = gru_model()
print(model_gru)

gru_model(
  (gru_layer): GRU(300, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=3, bias=True)
)


In [37]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
criterion = criterion
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.Adam(model_gru.parameters(), lr=0.001)

# number of epochs to train the model
n_epochs = 20

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity
best_acc = 0

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    # train the model #
    model_gru.train() # prep model for training
    for data, target in train_loader: # iterates upto number of batch size
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_gru(data)
        # calculate the loss
        loss = criterion(output, target-1)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
          
    # validate the model #
    model_gru.eval() # prep model for evaluation
    correct = 0
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_gru(data)
        # calculate the loss
        loss = criterion(output, target-1)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        ypred = output.argmax(dim = 1)
        correct += (ypred == target-1).float().sum()
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tEpoch Accuracy: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss,
        correct/len(test_loader.dataset)
        ))
    
model_gru.eval() # prep model for evaluation
main_tar = []
predss = []
with torch.no_grad():
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_gru(data)
        # calculate the loss
        loss = criterion(output, target-1)
        ypred = output.argmax(dim = 1)
        for i in np.array(target-1):
            main_tar.append(i)
        for j in np.array(ypred):
            predss.append(j)

Epoch: 1 	Training Loss: 1.060141 	Validation Loss: 0.932728 	Epoch Accuracy: 0.540667
Epoch: 2 	Training Loss: 0.879136 	Validation Loss: 0.856946 	Epoch Accuracy: 0.594833
Epoch: 3 	Training Loss: 0.828150 	Validation Loss: 0.816709 	Epoch Accuracy: 0.628917
Epoch: 4 	Training Loss: 0.801484 	Validation Loss: 0.798368 	Epoch Accuracy: 0.640417
Epoch: 5 	Training Loss: 0.783992 	Validation Loss: 0.787840 	Epoch Accuracy: 0.644833
Epoch: 6 	Training Loss: 0.774065 	Validation Loss: 0.780350 	Epoch Accuracy: 0.649417
Epoch: 7 	Training Loss: 0.763436 	Validation Loss: 0.781249 	Epoch Accuracy: 0.649167
Epoch: 8 	Training Loss: 0.754326 	Validation Loss: 0.779876 	Epoch Accuracy: 0.653250
Epoch: 9 	Training Loss: 0.747879 	Validation Loss: 0.773450 	Epoch Accuracy: 0.650917
Epoch: 10 	Training Loss: 0.741420 	Validation Loss: 0.777334 	Epoch Accuracy: 0.650500
Epoch: 11 	Training Loss: 0.734496 	Validation Loss: 0.769225 	Epoch Accuracy: 0.658000
Epoch: 12 	Training Loss: 0.729450 	Valid

In [38]:
print("Accuracy Value: GRU")
gru_report = classification_report(main_tar, predss, digits=6, output_dict=True)
print(gru_report['accuracy'])

Accuracy Value: GRU
0.6630833333333334


- Task 5(c): Considering a  LSTM unit cell.

In [39]:
class lstm_model(nn.Module):
    def __init__(self):
        super(lstm_model, self).__init__()
        self.lstm_layer = nn.LSTM(300, 20, batch_first = True)
        self.fc = nn.Linear(20,3)

    def forward(self, input):
        output = input.view(-1,20,300)
        output, hidden = self.lstm_layer(output)
        output=self.fc(output[:,-1,:])
        return output

model_lstm = lstm_model()
print(model_lstm)

lstm_model(
  (lstm_layer): LSTM(300, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=3, bias=True)
)


In [40]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
criterion = criterion
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)

# number of epochs to train the model
n_epochs = 20

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity
best_acc = 0

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    # train the model #
    model_lstm.train() # prep model for training
    for data, target in train_loader: # iterates upto number of batch size
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_lstm(data)
        # calculate the loss
        loss = criterion(output, target-1)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
          
    # validate the model #
    model_lstm.eval() # prep model for evaluation
    correct = 0
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_lstm(data)
        # calculate the loss
        loss = criterion(output, target-1)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        ypred = output.argmax(dim = 1)
        correct += (ypred == target-1).float().sum()
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tEpoch Accuracy: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss,
        correct/len(test_loader.dataset)
        ))
    
model_lstm.eval() # prep model for evaluation
main_tar = []
predss = []
with torch.no_grad():
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_lstm(data)
        # calculate the loss
        loss = criterion(output, target-1)
        ypred = output.argmax(dim = 1)
        for i in np.array(target-1):
            main_tar.append(i)
        for j in np.array(ypred):
            predss.append(j)


Epoch: 1 	Training Loss: 1.034733 	Validation Loss: 0.913043 	Epoch Accuracy: 0.554833
Epoch: 2 	Training Loss: 0.886737 	Validation Loss: 0.873409 	Epoch Accuracy: 0.586083
Epoch: 3 	Training Loss: 0.855134 	Validation Loss: 0.861392 	Epoch Accuracy: 0.601417
Epoch: 4 	Training Loss: 0.830644 	Validation Loss: 0.830864 	Epoch Accuracy: 0.625000
Epoch: 5 	Training Loss: 0.807800 	Validation Loss: 0.810819 	Epoch Accuracy: 0.636583
Epoch: 6 	Training Loss: 0.792577 	Validation Loss: 0.805850 	Epoch Accuracy: 0.639667
Epoch: 7 	Training Loss: 0.779837 	Validation Loss: 0.796382 	Epoch Accuracy: 0.644167
Epoch: 8 	Training Loss: 0.769117 	Validation Loss: 0.788636 	Epoch Accuracy: 0.648833
Epoch: 9 	Training Loss: 0.758789 	Validation Loss: 0.785727 	Epoch Accuracy: 0.644083
Epoch: 10 	Training Loss: 0.751966 	Validation Loss: 0.782061 	Epoch Accuracy: 0.653083
Epoch: 11 	Training Loss: 0.741923 	Validation Loss: 0.777455 	Epoch Accuracy: 0.653833
Epoch: 12 	Training Loss: 0.735822 	Valid

In [41]:
print("Accuracy Values for each class")
lstm_report = classification_report(main_tar, predss, digits=6, output_dict=True)
print(lstm_report['accuracy'])

Accuracy Values for each class
0.65575
